In [27]:
import pandas as pd

file_path = "Y:\\Eva de Vil - Algolia Results\\output.json"
df_json = pd.read_json(file_path)

desired_columns = ["title", "publish_date"]
df_json = df_json[desired_columns]
df_json['publish_date'] = pd.to_datetime(df_json['publish_date'])
df_json['publish_date'] = df_json['publish_date'].dt.date

print(df_json)

                                  title publish_date
0        This Was Meant To Be Temporary   2024-01-13
1                 So Weak, So Desperate   2024-01-12
2                   Habits For My Slave   2024-01-07
3               Your Sex Life This Year   2024-01-05
4                    My Loser Pump Slut   2023-12-02
..                                  ...          ...
764                  Sucker For My Tits   2021-11-13
765      Cruel Cuckolding: Eat His Load   2021-11-12
766        Gooner's Pussy Free Paradise   2021-10-31
767            Motivational Ass Worship   2021-10-30
768  Quick Fishnet Ass Worship (iPhone)   2018-01-02

[769 rows x 2 columns]


In [76]:
import os
import re
from datetime import datetime

directory_path = "Y:\\Culture\\Videos\\Performers\\Eva de Vil\\"
file_names = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
df_files = pd.DataFrame(file_names, columns=["raw_file_name"])

def extract_after_last_dash(raw_file_name):
    last_part = raw_file_name.rsplit('-', 1)[-1] if '-' in raw_file_name else raw_file_name
    return last_part.replace(" (STOLEN)", "").replace(".mp4", "").strip()

def parse_date_from_filename(filename):
    # Regular expression pattern for YYYY-mm-DD
    pattern = r'\d{4}-\d{2}-\d{2}'
    
    # Search for the pattern in the filename
    match = re.search(pattern, filename)
    
    # If a match is found, parse it into a date
    if match:
        date_str = match.group()
        date_object = datetime.strptime(date_str, '%Y-%m-%d')
        return date_object.date()  # Use .date() to get just the date part
    
    # If no date found, return None or handle as needed
    return None

df_files["file_name"] = df_files["raw_file_name"].apply(extract_after_last_dash)
df_files["release_date"] = df_files["raw_file_name"].apply(parse_date_from_filename)

print(df_files)

                                         raw_file_name                                          file_name release_date
0    Eva de Vil & Blonde Kitty - Double Domme Strap...                                  On Mouth Training         None
1    Eva de Vil & Princess Miki - Double Domme Blac...                             Double Domme Blackmail         None
2    Eva de Vil - 2017-10-29 - Sensual CEI Initiati...                 Sensual CEI Initiation For Goddess   2017-10-29
3    Eva de Vil - 2017-12-11 - Leather JOI (STOLEN)...                                        Leather JOI   2017-12-11
4    Eva de Vil - 2017-12-19 - You're Weak (STOLEN)...                                        You're Weak   2017-12-19
5    Eva de Vil - 2018-01-12 - Goddess Eva Ruins Yo...                        Goddess Eva Ruins Your Date   2018-01-12
6    Eva de Vil - 2018-01-24 - Christmas Titty Trea...                              Christmas Titty Treat   2018-01-24
7    Eva de Vil - 2018-01-27 - Sexy White Bikini

In [77]:
import Levenshtein

def levenshtein_ratio(name1, name2):
    return Levenshtein.ratio(name1, name2)

# Set a threshold for matching
threshold = 0.80  # You can adjust this value

# Compare each pair of rows
matches = []
for i, row1 in df_json.iterrows():
    for j, row2 in df_files.iterrows():
        ratio = levenshtein_ratio(row1["title"], row2["file_name"])
        if ratio >= threshold:
            matches.append((i, j, ratio))

# Create a new DataFrame based on matches
matched_rows = [(df_json.iloc[i], df_files.iloc[j], ratio) for i, j, ratio in matches]
df_matched = pd.DataFrame(matched_rows, columns=['JSON Row', 'Files Row', 'Ratio'])

In [82]:
df_matched_flat = pd.DataFrame({
    'raw_filename': df_matched['Files Row'].apply(lambda x: x['raw_file_name']),
    'title': df_matched['JSON Row'].apply(lambda x: x['title']),
    'publish_date': df_matched['JSON Row'].apply(lambda x: x['publish_date']),
    'file_name': df_matched['Files Row'].apply(lambda x: x['file_name']),
    'release_date': df_matched['Files Row'].apply(lambda x: x['release_date']),
    'ratio': df_matched['Ratio'],
})


In [83]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

print(df_matched_flat)

                                          raw_filename                                              title publish_date                                          file_name release_date     ratio
0       Eva de Vil - So Weak So Desperate (STOLEN).mp4                              So Weak, So Desperate   2024-01-12                               So Weak So Desperate         None  0.975610
1         Eva de Vil - My Loser Pump Slut (STOLEN).mp4                                 My Loser Pump Slut   2023-12-02                                 My Loser Pump Slut         None  1.000000
2         Eva de Vil - Stare into My Eyes (STOLEN).mp4                                 Stare into My Eyes   2023-12-01                                 Stare into My Eyes         None  1.000000
3             Eva de Vil - Cuck in a Cage (STOLEN).mp4                                     Cuck in a Cage   2023-11-25                                     Cuck in a Cage         None  1.000000
4                Eva de Vil - Simps

In [97]:
# Find rows where publish_date and release_date are different
different_dates = df_matched_flat[
    (df_matched_flat['release_date'].notna()) &
    ~(df_matched_flat['title'].str.startswith("Edge Slut Training"))
]

# Display these rows
print(different_dates)


                                          raw_filename                                              title publish_date                                          file_name release_date     ratio
8    Eva de Vil - 2023-11-04 - Blue Ball Fuel (STOL...                                     Blue Ball Fuel   2023-11-04                                     Blue Ball Fuel   2023-11-04  1.000000
9    Eva de Vil - 2023-11-03 - Playing Pegging Audi...                              Playing Pegging Audio   2023-11-03                              Playing Pegging Audio   2023-11-03  1.000000
25   Eva de Vil - 2023-09-15 - Forbidden Fruit (STO...                                    Forbidden Fruit   2023-09-15                                    Forbidden Fruit   2023-09-15  1.000000
26       Eva de Vil - 2023-09-15 - Forbidden Fruit.mp4                                    Forbidden Fruit   2023-09-15                                    Forbidden Fruit   2023-09-15  1.000000
29   Eva de Vil - 2023-05-13 - Myst

In [108]:
df_filtered = df_matched_flat.loc[
    ~df_matched_flat["release_date"].notna() &
    df_matched_flat["raw_filename"].str.contains("STOLEN")
]

print(df_filtered)

                                          raw_filename                                             title publish_date                                         file_name release_date     ratio
0       Eva de Vil - So Weak So Desperate (STOLEN).mp4                             So Weak, So Desperate   2024-01-12                              So Weak So Desperate         None  0.975610
1         Eva de Vil - My Loser Pump Slut (STOLEN).mp4                                My Loser Pump Slut   2023-12-02                                My Loser Pump Slut         None  1.000000
2         Eva de Vil - Stare into My Eyes (STOLEN).mp4                                Stare into My Eyes   2023-12-01                                Stare into My Eyes         None  1.000000
3             Eva de Vil - Cuck in a Cage (STOLEN).mp4                                    Cuck in a Cage   2023-11-25                                    Cuck in a Cage         None  1.000000
4                Eva de Vil - Simps Serve (ST

In [117]:
df_renames = pd.DataFrame({
    'old_filename': df_filtered['raw_filename'],
    'new_filename': "Eva de Vil - " + pd.to_datetime(df_filtered['publish_date']).dt.strftime('%Y-%m-%d') + " - " + df_filtered['title'] + " (STOLEN).mp4",
})

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

print(df_renames)

                                          old_filename                                       new_filename
0       Eva de Vil - So Weak So Desperate (STOLEN).mp4  Eva de Vil - 2024-01-12 - So Weak, So Desperat...
1         Eva de Vil - My Loser Pump Slut (STOLEN).mp4  Eva de Vil - 2023-12-02 - My Loser Pump Slut (...
2         Eva de Vil - Stare into My Eyes (STOLEN).mp4  Eva de Vil - 2023-12-01 - Stare into My Eyes (...
3             Eva de Vil - Cuck in a Cage (STOLEN).mp4  Eva de Vil - 2023-11-25 - Cuck in a Cage (STOL...
4                Eva de Vil - Simps Serve (STOLEN).mp4  Eva de Vil - 2023-11-24 - Simps Serve (STOLEN)...
5     Eva de Vil - Cleavage Cum Countdown (STOLEN).mp4  Eva de Vil - 2023-11-18 - Cleavage Cum Countdo...
6           Eva de Vil - The Moaning Game (STOLEN).mp4  Eva de Vil - 2023-11-11 - The Moaning Game (ST...
7          Eva de Vil - Late Night Edging (STOLEN).mp4  Eva de Vil - 2023-11-10 - Late Night Edging (S...
12               Eva de Vil - Lick My Abs (STO